In [85]:
import numpy as np
import tensorflow as tf
import keras
from keras.layers import BatchNormalization
import math

In [86]:
import sys
sys.path.append('../')

# Load test images
x_test_real = np.load("test_images/x_test_real.npy")
x_test_imag = np.load("test_images/x_test_imag.npy")

# Take first image and convert to tensor
# Expand first dim so shape is N,H,W,C_in
x_test_real = tf.expand_dims(tf.convert_to_tensor(x_test_real[0]), axis=0)
x_test_imag = tf.expand_dims(tf.convert_to_tensor(x_test_imag[0]), axis=0)

print("Shape of input feature map: ", x_test_real.shape)
print("Real Input: ", x_test_real[0][0][3])

print("\nImag Input: ", x_test_imag[0][7][15])


# Load filters/kernels weights already in freq domain
# filters shape: H,W,C_in,C_out
filters_l1_real = tf.convert_to_tensor(np.load("weight_files/conv_weights_real_layer1.npy"))
filters_l1_imag = tf.convert_to_tensor(np.load("weight_files/conv_weights_imag_layer1.npy"))

filters_l2_real = tf.convert_to_tensor(np.load("weight_files/conv_weights_real_layer2.npy"))
filters_l2_imag = tf.convert_to_tensor(np.load("weight_files/conv_weights_imag_layer2.npy"))

filters_l3_real = tf.convert_to_tensor(np.load("weight_files/conv_weights_real_layer3.npy"))
filters_l3_imag = tf.convert_to_tensor(np.load("weight_files/conv_weights_imag_layer3.npy"))

filters_l4_real = tf.convert_to_tensor(np.load("weight_files/conv_weights_real_layer4.npy"))
filters_l4_imag = tf.convert_to_tensor(np.load("weight_files/conv_weights_imag_layer4.npy"))

filters_l5_real = tf.convert_to_tensor(np.load("weight_files/conv_weights_real_layer5.npy"))
filters_l5_imag = tf.convert_to_tensor(np.load("weight_files/conv_weights_imag_layer5.npy"))

print("Shape of filters: ",filters_l1_imag.shape)
print(filters_l1_real[0][1][0])

b_norm_real = []
b_norm_imag = []
b_norm_variable_names = ["gamma", "beta", "moving_mean", "moving_var"]

# Load bnorm layer values
for layer in range(8):
    real_variables = {}
    imag_variables = {}
    for variable in b_norm_variable_names:
        filename = "weight_files/b_norm" + str(layer+1) + "_real_" + variable + ".npy" 
        real_variables[variable] = np.load(filename)
        filename = "weight_files/b_norm" + str(layer+1) + "_imag_" + variable + ".npy" 
        imag_variables[variable] = np.load(filename)
    b_norm_real.append(real_variables)
    b_norm_imag.append(imag_variables)

# Load Dense Layer Values
dense_real = []
dense_imag = []
dense_variable_names = ["kernel", "bias"]
for layer in range(3):
    real_variables = {}
    imag_variables = {}
    for variable in dense_variable_names:
        filename = "weight_files/dense" + str(layer+1) + "_real_" + variable + ".npy" 
        real_variables[variable] = np.load(filename)
        filename = "weight_files/dense" + str(layer+1) + "_imag_" + variable + ".npy"
        imag_variables[variable] = np.load(filename)
    dense_real.append(real_variables)
    dense_imag.append(imag_variables) 
    
# Load Output Dense Layer Values
output_dense_weights = {
    "kernel": np.load("weight_files/dense4_kernel.npy"),
    "bias": np.load("weight_files/dense4_bias.npy")
}


Shape of input feature map:  (1, 150, 150, 1)
Real Input:  tf.Tensor([-340.0954], shape=(1,), dtype=float32)

Imag Input:  tf.Tensor([13.17975], shape=(1,), dtype=float32)
Shape of filters:  (150, 150, 1, 32)
tf.Tensor(
[ 1.7070187  -0.6532587  -1.411036    0.49434805  0.14689273 -0.18017149
  0.62535787 -0.1872252   2.8218632   1.6008384   0.1738767  -1.2688624
  4.5345554  -1.607304   -0.9625048  -1.8037035   0.37793916 -0.7210022
  0.82449675  0.9351041   0.80745524  3.7595797   1.3952866   1.4047085
 -1.3126156  -0.72550225  0.3099211   0.5940683  -0.9378712   0.681558
  1.0259459  -1.9606347 ], shape=(32,), dtype=float32)


## First Network Block
Block:
- Elementwise Multiply
- Batch Normalize
- Activation
- Pooling

In [87]:
for i in range(8):
    print(b_norm_real[i]["beta"].shape)

(32,)
(32,)
(64,)
(64,)
(128,)
(512,)
(256,)
(256,)


In [88]:
# Initialize bnorm layers
b_norm_shapes = [
    (150,150,32), (75,75,32), (37,37,64), (18,18,64), (9,9,128), (512), (256), (256)
]

b_norm_layers_real = []
b_norm_layers_imag = []

for layer in range(8):
    real_layer = keras.Sequential([
        keras.layers.Input(b_norm_shapes[layer]),
        keras.layers.BatchNormalization()
    ])
    real_layer.layers[0].set_weights([
        b_norm_real[layer]["gamma"], b_norm_real[layer]["beta"],
        b_norm_real[layer]["moving_mean"], b_norm_real[layer]["moving_var"]
    ])
    b_norm_layers_real.append(real_layer)

    imag_layer = keras.Sequential([
        keras.layers.Input(b_norm_shapes[layer]),
        keras.layers.BatchNormalization()
    ])
    imag_layer.layers[0].set_weights([
        b_norm_imag[layer]["gamma"], b_norm_imag[layer]["beta"],
        b_norm_imag[layer]["moving_mean"], b_norm_imag[layer]["moving_var"]
    ])
    b_norm_layers_imag.append(imag_layer)



In [89]:
# Initialize Dense Layers
dense_units = [512, 256, 256]
dense_input_shapes = [(2048), (512), (256)]

dense_layers_real = []
dense_layers_imag = []

for layer in range(3):
    real_layer = keras.Sequential([
        keras.layers.Input(shape=dense_input_shapes[layer]),
        keras.layers.Dense(units=dense_units[layer])
    ])
    real_layer.layers[0].set_weights([
        dense_real[layer]["kernel"], dense_real[layer]["bias"]
    ])
    dense_layers_real.append(real_layer)

    imag_layer = keras.Sequential([
        keras.layers.Input(shape=dense_input_shapes[layer]),
        keras.layers.Dense(units=dense_units[layer])
    ])
    imag_layer.layers[0].set_weights([
        dense_imag[layer]["kernel"], dense_imag[layer]["bias"]
    ])
    dense_layers_imag.append(imag_layer)

In [90]:
# Output Dense Layer
output_dense_layer = keras.Sequential([
    keras.layers.Input(shape=(512)),
    keras.layers.Dense(units=3)
])
output_dense_layer.layers[0].set_weights([
    output_dense_weights["kernel"], output_dense_weights["bias"]
])

In [91]:
# Leaky ReLu
leaky_relu = keras.layers.LeakyReLU(alpha=0.2)
# Max Pool
max_pool = keras.layers.MaxPool2D(pool_size=(2,2))
# Flatten
flatten_layer = keras.layers.Flatten()
# Concat
concat_layer = keras.layers.Concatenate(axis=1)
# Softmax
softmax_activation = keras.layers.Activation(activation="softmax")

In [92]:
from fourier_model import element_wise_multiply

# Block 1
out_real, out_imag = element_wise_multiply(x_test_real, x_test_imag, filters_l1_real, filters_l1_imag)
out_real, out_imag = b_norm_layers_real[0].call(out_real, training=False), b_norm_layers_imag[0].call(out_imag, training=False)
out_real, out_imag = leaky_relu(out_real), leaky_relu(out_imag)
out_real, out_imag = max_pool(out_real), max_pool(out_imag)

# print("Shape of Block 1 Output: ", out_real.shape)
# print("\nReal F-Map Sample: ", out_real[0][15][12])
# print("\nImag F-Map Sample: ", out_imag[0][4][49])

# Block 2
out_real, out_imag = element_wise_multiply(out_real, out_imag, filters_l2_real, filters_l2_imag)
out_real, out_imag = b_norm_layers_real[1].call(out_real, training=False), b_norm_layers_imag[1].call(out_imag, training=False)
out_real, out_imag = leaky_relu(out_real), leaky_relu(out_imag)
out_real, out_imag = max_pool(out_real), max_pool(out_imag)


# Block 3
out_real, out_imag = element_wise_multiply(out_real, out_imag, filters_l3_real, filters_l3_imag)
out_real, out_imag = b_norm_layers_real[2].call(out_real, training=False), b_norm_layers_imag[2].call(out_imag, training=False)
out_real, out_imag = leaky_relu(out_real), leaky_relu(out_imag)
out_real, out_imag = max_pool(out_real), max_pool(out_imag)

# Block 4
out_real, out_imag = element_wise_multiply(out_real, out_imag, filters_l4_real, filters_l4_imag)
out_real, out_imag = b_norm_layers_real[3].call(out_real, training=False), b_norm_layers_imag[3].call(out_imag, training=False)
out_real, out_imag = leaky_relu(out_real), leaky_relu(out_imag)
out_real, out_imag = max_pool(out_real), max_pool(out_imag)

# Block 5
out_real, out_imag = element_wise_multiply(out_real, out_imag, filters_l5_real, filters_l5_imag)
out_real, out_imag = b_norm_layers_real[4].call(out_real, training=False), b_norm_layers_imag[4].call(out_imag, training=False)
out_real, out_imag = leaky_relu(out_real), leaky_relu(out_imag)
out_real, out_imag = max_pool(out_real), max_pool(out_imag)

# print("\nShape of Block 5 Output: ", out_real.shape)
# print("\nReal F-Map Sample: ", out_real[0][1][2])
# print("\nImag F-Map Sample: ", out_imag[0][2][3])

# FLATTEN
out_real, out_imag = flatten_layer(out_real), flatten_layer(out_imag)


# FC1
out_real, out_imag = dense_layers_real[0].call(out_real, training=False), dense_layers_imag[0].call(out_imag, training=False)
out_real, out_imag = b_norm_layers_real[5].call(out_real, training=False), b_norm_layers_imag[5].call(out_imag, training=False)
out_real, out_imag = leaky_relu(out_real), leaky_relu(out_imag)



# FC2
out_real, out_imag = dense_layers_real[1].call(out_real, training=False), dense_layers_imag[1].call(out_imag, training=False)
out_real, out_imag = b_norm_layers_real[6].call(out_real, training=False), b_norm_layers_imag[6].call(out_imag, training=False)
out_real, out_imag = leaky_relu(out_real), leaky_relu(out_imag)

# FC3
out_real, out_imag = dense_layers_real[2].call(out_real, training=False), dense_layers_imag[2].call(out_imag, training=False)
out_real, out_imag = b_norm_layers_real[7].call(out_real, training=False), b_norm_layers_imag[7].call(out_imag, training=False)
out_real, out_imag = leaky_relu(out_real), leaky_relu(out_imag)

print("\nShape of FC3 Output: ", out_real.shape)
print("\nReal F-Map Sample: ", out_real[0][255])
print("\nImag F-Map Sample: ", out_imag[0][240])

# CONCATENATE
out_concat = concat_layer([out_real, out_imag])

# OUTPUT
out_concat = output_dense_layer.call(out_concat, training=False)
probs = softmax_activation(out_concat)


Shape of FC3 Output:  (1, 256)

Real F-Map Sample:  tf.Tensor(0.7843845, shape=(), dtype=float32)

Imag F-Map Sample:  tf.Tensor(1.1313245, shape=(), dtype=float32)


In [105]:
out_concat

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[ 2.9407008,  4.036466 , -7.118903 ]], dtype=float32)>

In [97]:
probs

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[2.5053152e-01, 7.4945778e-01, 1.0715984e-05]], dtype=float32)>